In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM
from transformers import AutoConfig
import bigram
tokenizer = AutoTokenizer.from_pretrained("TinyPixel/small-llama2")
bigram_instance = bigram.bigram(tokenizer.vocab_size)

In [2]:
from datasets import load_dataset

# DatasetDict({
#     train: Dataset({
#         features: ['text'],
#         num_rows: 74 004 228
#     })
# })

# dataset = load_dataset("bookcorpus")
datasets = load_dataset("./bookcorpus-splitted")
datasets

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 59203382
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 14800846
    })
})

In [3]:
dataset_size = datasets['train'].num_rows
chunk_size = 10_000
for i in range(int(dataset_size / chunk_size) + 1):
    # select data samples, 选择部分数据，数据分块，处理大规模数据
    selected_data = datasets['train'].select(range(i * chunk_size, min((i + 1) * chunk_size, dataset_size)))
    # print(selected_data) # Dataset
    # print(type(selected_data['text'])) # list
    if i % 100 == 0:
        print(i * chunk_size)
        tokenizer = AutoTokenizer.from_pretrained("TinyPixel/small-llama2")
    plain_text = ' '.join(selected_data['text'])
    # print(len(plain_text))
    # tokenize
    ids = tokenizer(plain_text, truncation=False)
    # # print(len(ids['input_ids']))
    bigram_instance.add_and_count(ids['input_ids'])
    # break

bigram_probability = bigram_instance.fit()
'''
Debug:
# Error: When chunk_size equals 100,000, the program throws an error: 

Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...
thread '<unnamed>' panicked at /home/runner/work/tokenizers/tokenizers/tokenizers/src/models/bpe/word.rs:198:41:
index out of bounds: the len is 6163649 but the index is 274882678541
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace
# Solution: decrease chunk_size to 10,000

# Error: When chunk_size equals 10,000, the program throws an error: 
PanicException: index out of bounds: the len is 310549 but the index is 283468065632
# Restart the core

# Error:
PanicException: byte index 274878268401 is out of bounds of `▁dinner was slow cooking , mainly because they were all too listless and exhausted to mess with it . then you toil hard to make a hundred rupee . luke 's mother was at his side . `` that 's my specialty . '' `` it 's final . '' `` i 'm not ashamed of my `[...]
# Solution: reboot


'''

0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000
21000000
22000000
23000000
24000000
25000000
26000000
27000000
28000000
29000000
30000000
31000000
32000000
33000000
34000000
35000000
36000000
37000000
38000000
39000000
40000000
41000000
42000000
43000000
44000000
45000000
46000000
47000000
48000000
49000000
50000000
51000000
52000000
53000000
54000000
55000000
56000000
57000000
58000000
59000000


"\nDebug:\n# Error: When chunk_size equals 100,000, the program throws an error: \n\nOutput is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...\nthread '<unnamed>' panicked at /home/runner/work/tokenizers/tokenizers/tokenizers/src/models/bpe/word.rs:198:41:\nindex out of bounds: the len is 6163649 but the index is 274882678541\nnote: run with `RUST_BACKTRACE=1` environment variable to display a backtrace\n# Solution: decrease chunk_size to 10,000\n\n# Error: When chunk_size equals 10,000, the program throws an error: \nPanicException: index out of bounds: the len is 310549 but the index is 283468065632\n# Restart the core\n\n# Error:\nPanicException: byte index 274878268401 is out of bounds of `▁dinner was slow cooking , mainly because they were all too listless and exhausted to mess with it . then you toil hard to make a hundred rupee . luke 's mother was at his side . `` that 's my specialty . '' `` it 's final . '' `` i 'm not ashamed 

In [4]:
import numpy as np
# save data efficiently. @ How to Save a NumPy Array to File for Machine Learning
np.save('bigram_probability.npy', bigram_probability)

In [1]:
# load numpy array from npy file
from numpy import load
# load array
data = load('bigram_probability.npy')
# print the array
print(data.shape)

(32000, 32000)


In [2]:
import numpy as np
# While broadcasting,  (32000,) behaves like a row vector (1, 32000)
np.sum(data,axis=1)[:50]

array([0.        , 1.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.99999933, 0.999995  , 0.        ,
       0.        , 0.        , 0.99999991, 0.99999999, 0.        ,
       0.        , 0.99999995, 0.99999999, 0.        , 0.99999997,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])